## Implementing a ChatGPT App with LangChain from Scratch

This is part of my **"Learn LangChain, Pinecone & OpenAI: Build Next-Gen LLM Apps"** course.

https://www.udemy.com/course/master-langchain-pinecone-openai-build-llm-applications/?referralCode=4B17E3BD4CBBEA3B8321

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True) 

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate


llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[
#         SystemMessage(content="You are a chatbot having a conversation with a human."),
        SystemMessage(content='You respond only in German.'),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)
    

Your prompt:  Paris is ...




> Entering new LLMChain chain...
Prompt after formatting:
System: You respond only in German.
Human: Paris is ...

> Finished chain.
{'content': 'Paris is ...', 'text': 'Paris ist die Hauptstadt von Frankreich und eine der schönsten Städte der Welt. Es ist bekannt für seine beeindruckende Architektur, seine historischen Denkmäler wie den Eiffelturm und den Louvre sowie seine romantische Atmosphäre. Paris ist auch ein Zentrum für Kunst, Mode und Gastronomie und bietet eine Vielzahl von kulturellen Aktivitäten und kulinarischen Erlebnissen. Es ist definitiv ein Ort, den man in seinem Leben besuchen sollte.'}
--------------------------------------------------


Your prompt:  quit


Goodbye!


## Adding Chat Memory Using ConversationBufferMemory

In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=False) 

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain

# 1. Imports
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder


llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

# 2. Create memory 
memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True
)

# 3. add  MessagesPlaceholder(variable_name='messages') to the prompt
prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

# 4. Add the memory to the chain
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)
    

Your prompt:  Paris is ...


{'content': 'Paris is ...', 'chat_history': [HumanMessage(content='Paris is ...'), AIMessage(content='Paris is the capital city of France and is renowned for its art, culture, and history. It is famous for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. Paris is known as the "City of Love" and attracts millions of tourists each year. It is also famous for its gourmet cuisine, fashion industry, and charming neighborhoods.')], 'text': 'Paris is the capital city of France and is renowned for its art, culture, and history. It is famous for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. Paris is known as the "City of Love" and attracts millions of tourists each year. It is also famous for its gourmet cuisine, fashion industry, and charming neighborhoods.'}
--------------------------------------------------


Your prompt:  Its population is ...


{'content': 'Its population is ...', 'chat_history': [HumanMessage(content='Paris is ...'), AIMessage(content='Paris is the capital city of France and is renowned for its art, culture, and history. It is famous for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. Paris is known as the "City of Love" and attracts millions of tourists each year. It is also famous for its gourmet cuisine, fashion industry, and charming neighborhoods.'), HumanMessage(content='Its population is ...'), AIMessage(content='As of 2021, the estimated population of Paris is around 2.1 million people within the city limits. However, when considering the wider metropolitan area known as the Île-de-France region, the population increases to approximately 12.2 million people. Paris is one of the most populous cities in Europe and plays a central role in politics, economics, and culture in France and globally.')], 'text': 'As of 2021, the estimated population of Paris is around 2

Your prompt:  What about area?


{'content': 'What about area?', 'chat_history': [HumanMessage(content='Paris is ...'), AIMessage(content='Paris is the capital city of France and is renowned for its art, culture, and history. It is famous for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. Paris is known as the "City of Love" and attracts millions of tourists each year. It is also famous for its gourmet cuisine, fashion industry, and charming neighborhoods.'), HumanMessage(content='Its population is ...'), AIMessage(content='As of 2021, the estimated population of Paris is around 2.1 million people within the city limits. However, when considering the wider metropolitan area known as the Île-de-France region, the population increases to approximately 12.2 million people. Paris is one of the most populous cities in Europe and plays a central role in politics, economics, and culture in France and globally.'), HumanMessage(content='What about area?'), AIMessage(content='The city of

Your prompt:  What did I ask you?


{'content': 'What did I ask you?', 'chat_history': [HumanMessage(content='Paris is ...'), AIMessage(content='Paris is the capital city of France and is renowned for its art, culture, and history. It is famous for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. Paris is known as the "City of Love" and attracts millions of tourists each year. It is also famous for its gourmet cuisine, fashion industry, and charming neighborhoods.'), HumanMessage(content='Its population is ...'), AIMessage(content='As of 2021, the estimated population of Paris is around 2.1 million people within the city limits. However, when considering the wider metropolitan area known as the Île-de-France region, the population increases to approximately 12.2 million people. Paris is one of the most populous cities in Europe and plays a central role in politics, economics, and culture in France and globally.'), HumanMessage(content='What about area?'), AIMessage(content='The city

Your prompt:  quit


Goodbye!


## Saving Chat Sessions

In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=False) 

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain

# 1. Import FileChatMessageHistory
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

# 2. Add an additional keyword argument to the ConversationBufferMemory() constructor
history = FileChatMessageHistory('chat_history.json')
memory = ConversationBufferMemory(
    memory_key='chat_history',
    chat_memory=history,
    return_messages=True
)

prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), 
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)
    

Your prompt:  Earth mass is ...


{'content': 'Earth mass is ...', 'chat_history': [], 'text': "The Earth's mass is approximately 5.97 x 10^24 kilograms (or 1.32 x 10^25 pounds). It is the total amount of matter contained within the Earth."}
--------------------------------------------------


Your prompt:  what about the Moon and Sun?


{'content': 'what about the Moon and Sun?', 'chat_history': [HumanMessage(content='Earth mass is ...'), AIMessage(content="The Earth's mass is approximately 5.97 x 10^24 kilograms (or 1.32 x 10^25 pounds). It is the total amount of matter contained within the Earth.")], 'text': "The Moon has a mass of about 7.35 x 10^22 kilograms (or 1.62 x 10^23 pounds), which is about 1.2% of the Earth's mass. The Sun, on the other hand, has a much larger mass. Its mass is approximately 1.99 x 10^30 kilograms (or 4.39 x 10^30 pounds), which is about 332,900 times the mass of the Earth. The Sun's mass accounts for about 99.86% of the total mass of the entire solar system."}
--------------------------------------------------


Your prompt:  quit


Goodbye!


In [4]:
# The messages property contains the list of messages in order.
print(history.messages)

[HumanMessage(content='Earth mass is ...'), AIMessage(content="The Earth's mass is approximately 5.97 x 10^24 kilograms (or 1.32 x 10^25 pounds). It is the total amount of matter contained within the Earth."), HumanMessage(content='what about the Moon and Sun?'), AIMessage(content="The Moon has a mass of about 7.35 x 10^22 kilograms (or 1.62 x 10^23 pounds), which is about 1.2% of the Earth's mass. The Sun, on the other hand, has a much larger mass. Its mass is approximately 1.99 x 10^30 kilograms (or 4.39 x 10^30 pounds), which is about 332,900 times the mass of the Earth. The Sun's mass accounts for about 99.86% of the total mass of the entire solar system.")]
